In [15]:
!pip install nuscenes-devkit &> /dev/null

In [16]:
from nuimages import NuImages
import os
import json
from tqdm import tqdm
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from itertools import groupby
import numpy as np

In [17]:
dataroot = '/usr0/tma1/datasets/nuimages'

In [4]:
print('Creating category id files...')
category_file = os.path.join(dataroot, 'category_id.json')
if (not os.path.exists(category_file)):
    fd = open(os.path.join(dataroot, 'v1.0-train/category.json'), 'r')
    category = json.load(fd)
    category_id = dict()
    
    for i, cat in enumerate(category):
        category_id[cat['name']] = i
    
    fd2 = open(category_file, 'w+')
    json.dump(category_id, fd2)
    
    fd.close()
    fd2.close()
print('Done!')

Creating category id files...
Done!


In [5]:
category_id_file = os.path.join(dataroot, 'category_id.json')
fd = open(category_id_file, 'r')
category_id = json.load(fd)
fd.close()

In [64]:
def object_ann_detectron_format(nuim, sample):
    data = dict()
    
    # Get sample
    sd = nuim.get('sample_data', sample['key_camera_token'])
    data['file_name'] = os.path.join(dataroot, sd['filename'])
    data['height'] = sd['height']
    data['width'] = sd['width']
    data['image_id'] = sample['key_camera_token']
    
    annotations = []
    obj_anns, _ = nuim.list_anns(sample['token'], verbose=False)

    for obj_ann_token in obj_anns:
        ann = dict()
        obj_ann = nuim.get('object_ann', obj_ann_token)
        ann['bbox'] = obj_ann['bbox']
        ann['bbox_mode'] = 0 # detectron2.structures.BoxMode.XYXY_ABS
        ann_category = nuim.get('category', obj_ann['category_token'])
        ann['category_id'] = category_id[ann_category['name']]

        annotations.append(ann)
    data['annotations'] = annotations
    return data


Save the annotations of each split into detectron input data format
Test has no data because no annotations. Below is generated based on annotations

In [65]:
splits = ['v1.0-train', 'v1.0-val', 'v1.0-test']
for split in splits:
    print("Generating detectron data for {}".format(split))
    split_data = []
    nuim = NuImages(dataroot=dataroot, version=split, verbose=True, lazy=True)
    
    for s in tqdm(nuim.sample):
        data = object_ann_detectron_format(nuim, s)
        split_data.append(data)
    
#     grouped_anns = groupby(nuim.object_ann, lambda k: k['sample_data_token'])
#     for k, v in tqdm(grouped_anns):
#         sd_token = list(v)[0]['sample_data_token']
#         object_anns = [o for o in nuim.object_ann if o['sample_data_token'] == sd_token]
#         print("num objects", len(object_anns))
#         nuim.render_image(sd_token, annotation_type='all',
#                           with_category=True, with_attributes=True, box_line_width=-1, render_scale=5)
#         data = object_ann_detectron_format(nuim, k, v)
#         split_data.append(data)
#         break
#     break
    
    filename = os.path.join(os.path.join(dataroot, 'detectron_data'), split + '_detectron.json')
    fd = open(filename, 'w+')
    json.dump(split_data, fd)
    fd.close()
    print("... done")

Generating detectron data for v1.0-train
Loading nuImages tables for version v1.0-train...
Done loading in 0.000 seconds (lazy=True).
Loaded 67279 sample(s) in 1.015s,


  0%|          | 0/67279 [00:00<?, ?it/s]

Loaded 872181 sample_data(s) in 43.912s,
Loaded 557715 object_ann(s) in 25.733s,


  0%|          | 1/67279 [01:24<1575:08:48, 84.29s/it]

Loaded 73755 surface_ann(s) in 13.788s,
Loaded 25 category(s) in 0.016s,


...

Loaded 17995 surface_ann(s) in 0.892s,
Loaded 25 category(s) in 0.013s,


100%|██████████| 16445/16445 [02:52<00:00, 95.12it/s]


... done
Generating detectron data for v1.0-test
Loading nuImages tables for version v1.0-test...
Done loading in 0.000 seconds (lazy=True).
Loaded 9752 sample(s) in 0.011s,


100%|██████████| 9752/9752 [00:00<00:00, 12820.91it/s]

Loaded 126276 sample_data(s) in 0.661s,
Loaded 0 object_ann(s) in 0.000s,
Loaded 0 surface_ann(s) in 0.000s,
Loaded 25 category(s) in 0.002s,
... done


Filter for traffic cones only

In [6]:
print(category_id)
traffic_cone_id = 12
# movable_object.trafficcone: 12

{'animal': 0, 'flat.driveable_surface': 1, 'human.pedestrian.adult': 2, 'human.pedestrian.child': 3, 'human.pedestrian.construction_worker': 4, 'human.pedestrian.personal_mobility': 5, 'human.pedestrian.police_officer': 6, 'human.pedestrian.stroller': 7, 'human.pedestrian.wheelchair': 8, 'movable_object.barrier': 9, 'movable_object.debris': 10, 'movable_object.pushable_pullable': 11, 'movable_object.trafficcone': 12, 'static_object.bicycle_rack': 13, 'vehicle.bicycle': 14, 'vehicle.bus.bendy': 15, 'vehicle.bus.rigid': 16, 'vehicle.car': 17, 'vehicle.construction': 18, 'vehicle.ego': 19, 'vehicle.emergency.ambulance': 20, 'vehicle.emergency.police': 21, 'vehicle.motorcycle': 22, 'vehicle.trailer': 23, 'vehicle.truck': 24}


In [69]:
splits = ['v1.0-train', 'v1.0-val', 'v1.0-test']
for split in splits:
    print("Filter detectron data for {}".format(split))
    path = os.path.join(os.path.join(dataroot, 'detectron_data'), split + '_detectron.json')
    fd = open(path, 'r')
    data = json.load(fd)
    
    filtered_data = []
    for d in data:
        filtered_anns = []
        d_anns = d['annotations']
            
        for ann in d_anns:
            if ann['category_id'] == traffic_cone_id:
                ann['category_id'] = 0
                filtered_anns.append(ann)
        
        if (len(filtered_anns) > 0):
            d['annotations'] = filtered_anns
            filtered_data.append(d)
    
    path = os.path.join(os.path.join(dataroot, 'detectron_data'), split + '_trafficcone_detectron.json')
    fd2 = open(path, 'w+')
    json.dump(filtered_data, fd2)
    
    fd.close()
    fd2.close()
    print("... done")

Filter detectron data for v1.0-train
... done
Filter detectron data for v1.0-val
... done
Filter detectron data for v1.0-test
... done


In [10]:
def get_box_size(bbox):
    return abs(bbox[0] - bbox[2]) * abs(bbox[1] - bbox[3])

In [11]:
# Get counts
splits = ['v1.0-train', 'v1.0-val', 'v1.0-test']
for split in splits:
    print("Filter detectron data for {}".format(split))
    path = os.path.join(os.path.join(dataroot, 'detectron_data'), split + '_detectron.json')
    fd = open(path, 'r')
    data = json.load(fd)
    print(split, len(data))
    small, medium, large = 0, 0, 0
    for d in data:
        for ann in d['annotations']:
            if get_box_size(ann['bbox']) <= (32**2):
                small += 1
            elif get_box_size(ann['bbox']) <= (96**2):
                medium += 1
            else:
                large += 1
    print(split, "small", small)
    print(split, "medium", medium)
    print(split, "large", large)
    fd.close()

Filter detectron data for v1.0-train
v1.0-train 67279
v1.0-train small 157619
v1.0-train medium 271410
v1.0-train large 128686
Filter detectron data for v1.0-val
v1.0-val 16445
v1.0-val small 40048
v1.0-val medium 65547
v1.0-val large 30479
Filter detectron data for v1.0-test
v1.0-test 9752
v1.0-test small 0
v1.0-test medium 0
v1.0-test large 0


In [37]:
# Split validation data into validation and test
path = os.path.join(os.path.join(dataroot, 'detectron_data'), 'v1.0-val_trafficcone_detectron.json')
print(path)
fd = open(path, 'r')
data = json.load(fd)
print(len(data))
num_instances = len(data)
indices = np.arange(num_instances)
np.random.shuffle(indices)
split_index = int(num_instances / 2)

data_val = list(np.array(data)[indices[:split_index]])
data_test = list(np.array(data)[indices[split_index:]])

val_path = os.path.join(os.path.join(dataroot, 'detectron_data'), 'val_val_trafficcone_detectron.json')
test_path = os.path.join(os.path.join(dataroot, 'detectron_data'), 'val_test_trafficcone_detectron.json')

fd_val = open(val_path, 'w+')
json.dump(data_val, fd_val)

fd_test = open(test_path, 'w+')
json.dump(data_test, fd_test)

fd_val.close()
fd_test.close()
fd.close()

#Done!

/usr0/tma1/datasets/nuimages/detectron_data/v1.0-val_trafficcone_detectron.json
4526


In [39]:
val_path = os.path.join(os.path.join(dataroot, 'detectron_data'), 'val_val_trafficcone_detectron.json')
fd = open(val_path, 'r')
data = json.load(fd)
fd.close()

val_path_data = os.path.join(os.path.join(dataroot, 'detectron_data'), 'v1.0-val_val_trafficcone_detectron.json')
val_data = dict()
for d in data:
    val_data[d['image_id']] = d

fd = open(val_path_data, 'w+')
json.dump(json.dumps(val_data), fd)

test_path = os.path.join(os.path.join(dataroot, 'detectron_data'), 'val_test_trafficcone_detectron.json')
fd = open(test_path, 'r')
data = json.load(fd)
fd.close()

test_path_data = os.path.join(os.path.join(dataroot, 'detectron_data'), 'v1.0-val_test_trafficcone_detectron.json')
test_data = dict()
for d in data:
    test_data[d['image_id']] = d

fd = open(test_path_data, 'w+')
json.dump(json.dumps(test_data), fd)